In [1]:
#first import needed libraries
import numpy as np # numpy arrays
import pandas as pd # dataframes and many many other things
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # work with json files
import requests #make requests to urls
import urllib #open urls

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from bs4 import BeautifulSoup

# Task 1
So, we're opening using library urllib desired wikipedia website, which has the target table. <br>
After that, we're transferring html file derived into beatiful soup file. <br>
The last this is we use prettify method to make the file readable.

In [2]:
url_wiki='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html=urllib.request.urlopen(url_wiki).read()
wiki_soup=BeautifulSoup(wiki_html,'html.parser')
#print(wiki_soup.prettify())

### I really don't know how html works, so I just looked at the whole text at first (currently commented, because it's very long)
When looking at the file I've done text 'ctrl + F' search, how the table is coded in html, I've searched for the text words that was previous to the table ("here are no rural FSAs in Toronto") so, below the text part we fould the name ( table class="wikitable sortable" )

In [3]:
soup_table = wiki_soup.find('table',{'class':'wikitable sortable'})
#so then, after we locate the table, we only take that relevant part, and find every text line by searching 'td'
soup_links=soup_table.findAll('td')
columns=['Postcode', 'Borough', 'Neighbourhood']
postal_codes=pd.DataFrame(columns=columns)
temp_dict={}
for n,line in enumerate(soup_links):
#basically, every element in 'soup_links' is every cell in the table, and as we know there are 3 elements in every row, so we are configuring this algorithm to
#re-write a dictionary 'temp_dict' on every 3-rd iteration and append it to the created dataframe 'postal_codes'
    curr=line.get_text().strip('\n') # at this line - we're getting only relevant text without any html symbols and other things, also we're stripping the next line symbol
#this if is served for multi-purpose 1. if n%3==0 then the we've got the full temp_dict recorded that we need to record
# however if n==0, then we're at the start of the loop, and we don't have anything to write yet
# also it is given at the task that we need to delete all unassigned lists
    if (n%3==0) and (n!=0) and (temp_dict['Borough']!='Not assigned'):
# next if is to see when the postcode already had been recorded, so if the loc method with condition inside shows that it's empty - then we need to write a new line
        if postal_codes.loc[postal_codes['Postcode']==temp_dict['Postcode']].empty:
            postal_codes=postal_codes.append(temp_dict,ignore_index=True)
        else:
# if the postcode already had been recorded - then let's append things to it! (for convinience, i've put long condition as 'cond')
            cond=postal_codes['Postcode']==temp_dict['Postcode']
            postal_codes.loc[cond,'Neighbourhood']=postal_codes.loc[cond,'Neighbourhood'].astype(str)+', '+temp_dict['Neighbourhood']
    temp_dict[columns[n%3]]=curr
    
# if a neighbourhood not assigned, then it must be replaced
postal_codes.loc[postal_codes['Neighbourhood']=='Not assigned','Neighbourhood']=postal_codes.loc[postal_codes['Neighbourhood']=='Not assigned','Borough']
postal_codes.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [4]:
postal_codes.shape

(103, 3)

# Task 2
I am planning to use CanadaPost instead of google for geocoder, I registired on their website for this purposes <br>
link on the geocoder description of available providers https://geocoder.readthedocs.io/providers/CanadaPost.html, however apparently it has no location data present <br>
so then, after trying almost every algorithm there is, we find that geocodefarm is by far the best one

In [5]:
!pip install geocoder
import geocoder # import geocoder
#key_canadapost='<9fbd63fd35fb658f : 76c37056ae1906eef9197d>'
loc_file_url='http://cocl.us/Geospatial_data'

In [6]:
import time
# initialize your variable to None

# define a funtion that will use the geocoder and loop until you get the coordinates, or until half a second passes (2 iterations with 0.25 wait time)
def geocoder_finder(timestamp):
    i=0
    lat_lng_coords = None
    for i in range(2):
        #g = geocoder.canadapost('{}, Toronto, Ontario'.format(timestamp)) commented because canadapost apparently not showing any geographical coordinates
        g = geocoder.geocodefarm('{}, Toronto, Ontario'.format(timestamp))
        lat_lng_coords = g.latlng
        time.sleep(0.25) # sleep time had been introduced because a lot of API's are asynchronic - and also have a limit of many requests knocking on their door
        if lat_lng_coords is not None:
            return lat_lng_coords
    return None

#### This part is loading the backup data - in case our geocoder goes wrong

In [7]:
geo_file=pd.read_csv(loc_file_url)
geo_file.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Finally, we start a cycle with postcodes and gather the latitudes

In [9]:
postal_codes['Latitude']='NA'
postal_codes['Longitude']='NA'
postal_codes['Source']='NA'
# apart from Latitude and Longitude for each column, we're adding a source - notion, where the geocoordinates had been taken from
# unfortunately, geofarm free provides only 250 requests per day for a free plan, so if I make 2 requests and have some error with this dataset - I won't be able to get their data on my 3rd attempt
for i,st in enumerate(postal_codes['Postcode'].values):
    latlong=geocoder_finder(st)
    if latlong==None:
        latlong=geo_file.loc[geo_file['Postal Code']==st,['Latitude','Longitude']].values[0]
        postal_codes.loc[postal_codes['Postcode']==st,['Source']]='Provided file'
    else:
        postal_codes.loc[postal_codes['Postcode']==st,['Source']]='Geodecodefarm'
    postal_codes.loc[postal_codes['Postcode']==st,['Latitude','Longitude']]=latlong
    #print(latlong)

postal_codes

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Source
0,M3A,North York,Parkwoods,43.7513,-79.3299,Geodecodefarm
1,M4A,North York,Victoria Village,43.73,-79.3142,Geodecodefarm
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6552,-79.362,Geodecodefarm
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7228,-79.4507,Geodecodefarm
4,M7A,Queen's Park,Queen's Park,43.6645,-79.393,Geodecodefarm
5,M9A,Etobicoke,Islington Avenue,43.6627,-79.5284,Geodecodefarm
6,M1B,Scarborough,"Rouge, Malvern",43.8102,-79.1946,Geodecodefarm
7,M3B,North York,Don Mills North,43.7491,-79.362,Geodecodefarm
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.7076,-79.3109,Geodecodefarm
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6575,-79.3777,Geodecodefarm
